## `검색어 검색` 수집

# Initial Parameters 세팅

In [12]:
url_default = "https://search.naver.com/search.naver"

In [570]:
url_pge = 'https://s.search.naver.com/p/blog/search.naver'

In [365]:
import requests as req, json
from bs4 import BeautifulSoup as bs
from parsel import Selector
from urllib.parse import urljoin
from urllib import parse
import pandas as pd
from pprint import pprint
from datetime import date 
import re

In [10]:
headers_default = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}

In [644]:
params_default = {'where': 'blog',
    'query': '',
    'sm': 'tab_opt',
    'nso': 'so:r,p:all',
}

In [654]:
params_pge = {
    'where': 'blog',
    'sm': 'tab_pge',
    'api_type': '1',
    'query': '',
    'rev': '44',
    'start': '0',
    'dup_remove': '1',
    'post_blogurl':'',
    'post_blogurl_without':'',
    'nso':'so:r,p:all',
    'nlu_query': {"r_category": "32"},
    'dkey': '0',
    'source_query':'',
    'nx_search_query': '',
    'spq': '0',
    '_callback': 'viewMoreContents'
}

In [647]:
res_default = req.get(url_default, headers=headers_default, params= params_default)
soup = bs(res_default.text, 'html.parser')
option = soup.find('ul', {'class': 'lst_option'})
#pd
time_list  = {'전체': '0','1시간': '7' ,'2시간': '8','3시간': '9' ,'4시간': '10' ,'5시간': '11' ,'6시간': '12' ,'1일': '4', '1주': '1' , '1개월':'2', '3개월': '13','6개월': '6', '1년': '5', 
           '직접입력': '3'}

In [648]:
parsel = Selector(res_default.text)

# GET CATEGORY 부분

In [649]:
# get order category
order_names = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div/div[2]/ul/li[2]/div/div/a/text()').getall()
order_ids  = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div/div[2]/ul/li[2]/div/div/a/@data-param').getall()
order_list = []
for idx in range(len(order_names)): 
    tmp = {}
    tmp['order_name'] = order_names[idx]
    tmp['order_id']= order_ids[idx][4:]
    order_list.append(tmp)

order_list

[{'order_name': '관련도순', 'order_id': 'so:r,p:all'},
 {'order_name': '최신순', 'order_id': 'so:dd,p:all'}]

In [650]:
#get time category 
time_names = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div/div[2]/ul/li[3]/div/div[1]/a/text()').getall()
time_ids = parsel.xpath('/html/body/div[3]/div[2]/div/div[1]/div/div[2]/ul/li[3]/div/div[1]/a/@data-param').getall()
time_list = []
print(time_ids)
for idx in range(len(time_names)): 
    tmp = {} 
    tmp['time_name'] = time_names[idx]
    if idx == len(time_names)-1:
        tmp['time_id'] = time_ids[0][4:]
    else:
        tmp['time_id'] = time_ids[idx][4:]
    time_list.append(tmp)

time_list


['nso=so:r,p:all', 'nso=so:r,p:1h', 'nso=so:r,p:1d', 'nso=so:r,p:1w', 'nso=so:r,p:1m', 'nso=so:r,p:3m', 'nso=so:r,p:6m', 'nso=so:r,p:1y']


[{'time_name': '전체', 'time_id': 'so:r,p:all'},
 {'time_name': '1시간', 'time_id': 'so:r,p:1h'},
 {'time_name': '1일', 'time_id': 'so:r,p:1d'},
 {'time_name': '1주', 'time_id': 'so:r,p:1w'},
 {'time_name': '1개월', 'time_id': 'so:r,p:1m'},
 {'time_name': '3개월', 'time_id': 'so:r,p:3m'},
 {'time_name': '6개월', 'time_id': 'so:r,p:6m'},
 {'time_name': '1년', 'time_id': 'so:r,p:1y'},
 {'time_name': '직접입력', 'time_id': 'so:r,p:all'}]

In [651]:
# get corresponding parameters 
def option_params(params_initial, order, time, date = ""): 
    params_tmp = {}
    params_tmp = params_initial.copy() 
    #order
    for i in order_list: 
        if i['order_name'] == order: 
            params_tmp['nso'] = i['order_id'].split(',')[0]
    print(params_tmp['nso'])
    #time
    for j in time_list: 
        if j['time_name'] == time: 
            if time == '직접입력':
                ds, de = date['ds'], date['de']
                print(ds)
                ds = re.sub('[.]', '', ds)
                de = re.sub('[.]', '', de)
                print(ds)
                params_tmp['nso'] = params_tmp['nso'] + f", p:from{ds}to{de}"
            else:
                params_tmp['nso'] = params_tmp['nso'] + ',' + j['time_id'].split(',')[1]
    print(params_tmp['nso'])
    
    return params_tmp   

# USER INIPUTS, 

In [657]:
# User Inputs
query = "코로나"
order_option = "관련도순"
time_option = "1시간"
# optional: 직접 입력 date
# date = {'ds': "2021.07.06", 'de': "2021.07.12"}

page_number = 8 
data_category = ['기사제목']

In [727]:
params_search = {}
if page_number == 1:
    params_search = option_params(params_default, order_option, time_option, date)
    params_search['query'] = query
else: 
    params_search  = option_params(params_pge, order_option, time_option, date)
    params_search['query'] = query
    params_search['nx_search_query'] = query
    


so:r
so:r,p:1h


# Get Blog List

In [665]:
def get_blog_list(params_, p_number = 1): 
    params_pge = params_.copy() 
    blog_list = []
    if p_number == 1: 
        res = req.get(url_default, headers = headers_default, params = params_search)
        soup = bs(res.text, 'html.parser')
        blog_list = soup.find('ul', {'class': 'lst_total'}).find_all('li')
    else: 
        for p in range(p_number):
            params_pge['start'] =str(1+30*p)
            res2 = req.get(url_pge, headers = headers_default, params = params_pge)
            soup = bs(res2.text, 'html.parser')
            string1 = res2.text
            string2 = string1.split('viewMoreContents(')[1].split(')\n')[0] 
            js1 = json.loads(string2)  
            soup1 = bs(js1['html'], 'html.parser')
            blog_list.extend(soup1.find_all('div', {'class': 'total_area'})) 
    
    return blog_list 

In [677]:
blog_url_list = []
count = 0 
for blog in blog_list: 
    blog_url_list.append(blog.find('a', {'class': 'total_tit'})['href']) 
    print(blog.find('a', {'class': 'total_tit'}).text)
    count +=1
blog_url_list

코로나 여행 통영 섬추천 욕지도
유림목재 코로나19 점검일지 _ 7월 셋째주 (역학조사)
코로나 국민지원금, 얼마나 받나···
"배추 수확할 사람이 없어요" 코로나로 외국인근로자 부족
[육아/쇼핑] 코로나 시대, 무더위 여름 탈출 셀프... 
김근태 화백, ‘나는 꽃 너는 향’ 코로나 시대 위로 특별전
주전 : 주식전쟁 l 포스트 코로나 투자의 방향을 예상한다
끝 없는 코로나 시대 속 끊임없이 셀프 희망고문 하고 있는... 
코로나 격리자 생활지원비 신청
코로나 백신 시노백 2차 접종 완료
드디어 코로나 19 예방접종합니다.
여름휴가 계획은 코로나 지역별 거리두기 단계확인 후 세우세요!
평택시, ‘코로나19는 백신으로! 이웃사랑은 나눔으로!’
국민의 힘, 코로나 시국에 술판 로비연류의혹
코로나 PCR 검사를 마치고 왔습니다^^
코로나19 백신 - 모더나 화이자 비교
코로나 4단계 방역 연장 / 어린이집 학원 기준 / 여름방학 휴가... 
제9회 건강한 사회 질서 지키기 공모전 주제는 코로나19... 
코로나 위기에서 성장하는 외식기업의 공통점
코로나19 검사 백신접종
모더나 코로나19백신과 안면 필러시술 외
코로나 속 여름휴가 (변산반도)
제주코로나 생신파티, 생신케익주문
7월 셋째주 소소한 일상♡ 포르쉐, 폭염서울랜드, 2번째코로나... 
지창욱 코로나19 양성 판정 치료 확진
코로나19 피해 지원 국민지원금 지급 대상 확정!
30대 코로나 백신 접종시기는? 코로나가 턱 밑까지 온 느낌
코로나로 나타난 우울. 수어통역상담으로 회복 지원!!!
코로나 음식점 영업시간 거리두기 4단계 적용 시 기준은?
코로나 백신 접종에 대해서 반드시 아셔야 할 점들
백신 조롱곡 부르던 美방송인, 코로나 걸리자 "백신 맞아라"
[펌] CDC, 코로나19 테스트에 사용된 PCR 사용허가 취소
07.26. 코로나 현황
<서울시의 부평구 내 코로나19 생활치료센터 설치 철회 촉구>
제주시 그랜드보청기 코로나19백신 2차 접종 완료 안심 센터
아산시, 사업장 코로나19 자체접종 현장교육

['https://blog.naver.com/obh1680?Redirect=Log&logNo=222445987792',
 'https://blog.naver.com/woodstore?Redirect=Log&logNo=222445987633',
 'https://blog.naver.com/nmii2000?Redirect=Log&logNo=222445990706',
 'https://blog.naver.com/kssil1004?Redirect=Log&logNo=222445988154',
 'https://blog.naver.com/jeong620?Redirect=Log&logNo=222446005281',
 'https://blog.naver.com/gktkrk?Redirect=Log&logNo=222445983097',
 'https://blog.naver.com/online_kyobo?Redirect=Log&logNo=222445980039',
 'https://blog.naver.com/taehu12?Redirect=Log&logNo=222446008189',
 'https://blog.naver.com/leh7284?Redirect=Log&logNo=222446006598',
 'https://blog.naver.com/entercode?Redirect=Log&logNo=222445979939',
 'https://blog.naver.com/grace_no3?Redirect=Log&logNo=222446000036',
 'https://blog.naver.com/mjpiglet?Redirect=Log&logNo=222445955762',
 'https://blog.naver.com/iptnews?Redirect=Log&logNo=222445994913',
 'https://blog.naver.com/bluemenyes?Redirect=Log&logNo=222445980083',
 'https://blog.naver.com/kings-english?Redir

In [678]:
len(blog_url_list )

240

# Main Scraping Function


In [699]:
url_read = 'https://blog.naver.com/PostView.naver'
final_data = []
for url_ in blog_url_list: 
    blog_id = url_.split('?Redirect')[0].split('.com/')[1]
    logNo = url_.split('logNo=')[1]
    print(blog_id, logNo)
    params_tmp = {}
    params_tmp['blogId'] = blog_id
    params_tmp['logNo'] = logNo
    blog_info = get_info(url_read, params_tmp)
    final_data.append(blog_info)
blog_info
    


obh1680 222445987792
https://blog.naver.com/PostView.naver?blogId=obh1680&logNo=222445987792
woodstore 222445987633
https://blog.naver.com/PostView.naver?blogId=woodstore&logNo=222445987633
nmii2000 222445990706
https://blog.naver.com/PostView.naver?blogId=nmii2000&logNo=222445990706
kssil1004 222445988154
jeong620 222446005281
https://blog.naver.com/PostView.naver?blogId=jeong620&logNo=222446005281
gktkrk 222445983097
https://blog.naver.com/PostView.naver?blogId=gktkrk&logNo=222445983097
online_kyobo 222445980039
https://blog.naver.com/PostView.naver?blogId=online_kyobo&logNo=222445980039
taehu12 222446008189
https://blog.naver.com/PostView.naver?blogId=taehu12&logNo=222446008189
leh7284 222446006598
https://blog.naver.com/PostView.naver?blogId=leh7284&logNo=222446006598
entercode 222445979939
https://blog.naver.com/PostView.naver?blogId=entercode&logNo=222445979939
grace_no3 222446000036
mjpiglet 222445955762
https://blog.naver.com/PostView.naver?blogId=mjpiglet&logNo=222445955762
ip

댓글을 허용하지 않는 포스트 입니다.
bbr100003 222446005332
https://blog.naver.com/PostView.naver?blogId=bbr100003&logNo=222446005332
hsjk1004 222445982958
https://blog.naver.com/PostView.naver?blogId=hsjk1004&logNo=222445982958
randomp 222445985580
https://blog.naver.com/PostView.naver?blogId=randomp&logNo=222445985580
hyokumi 222445978329
https://blog.naver.com/PostView.naver?blogId=hyokumi&logNo=222445978329
pppiao 222445885860
https://blog.naver.com/PostView.naver?blogId=pppiao&logNo=222445885860
elena1210 222446008798
https://blog.naver.com/PostView.naver?blogId=elena1210&logNo=222446008798
tndustndud 222445978869
https://blog.naver.com/PostView.naver?blogId=tndustndud&logNo=222445978869
jeongda1221 222446000384
https://blog.naver.com/PostView.naver?blogId=jeongda1221&logNo=222446000384
miraov 222446006267
https://blog.naver.com/PostView.naver?blogId=miraov&logNo=222446006267
jjin2385 222445984715
https://blog.naver.com/PostView.naver?blogId=jjin2385&logNo=222445984715
lillijy00 222445991331
http

https://blog.naver.com/PostView.naver?blogId=skh1970kr&logNo=222445977705
0uizi0 222445990651
https://blog.naver.com/PostView.naver?blogId=0uizi0&logNo=222445990651
soccer30- 222445982270
https://blog.naver.com/PostView.naver?blogId=soccer30-&logNo=222445982270
haynghee2 222445984941
https://blog.naver.com/PostView.naver?blogId=haynghee2&logNo=222445984941
heeve1y 222445988946
uwoonli 222445992901
https://blog.naver.com/PostView.naver?blogId=uwoonli&logNo=222445992901
mktr1 222445973506
https://blog.naver.com/PostView.naver?blogId=mktr1&logNo=222445973506
iptnews 222445994460
https://blog.naver.com/PostView.naver?blogId=iptnews&logNo=222445994460
magicksj1 222445984451
https://blog.naver.com/PostView.naver?blogId=magicksj1&logNo=222445984451
campingcity3 222446000502
https://blog.naver.com/PostView.naver?blogId=campingcity3&logNo=222446000502
01073999832 222445985132
https://blog.naver.com/PostView.naver?blogId=01073999832&logNo=222445985132
psyoursummer 222445984549
https://blog.naver

{'제목': '혼자즐기는 일상 (feat. 먹고놀계소모임)',
 '작성자': '예측',
 '발행 시각': '1시간 전',
 '본문': '\n\n\n\n\n#먹고놀계#소모임#그때그때엔빵#계곡#쌍용계곡#물놀이#돼지막창#목살#삼겹살\u200b\u200b하위!\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n주말을 맞아 계곡을 두탕 뛰고왔어요!\u200b금요일날 한곳과 일요일날 한곳!\u200b일요일은 사진이 없어서 패스하고\u200b토요일 갔었던 계곡 일상입니다 ㅎ\u200b저는 먹고놀계 라는 고등학교 친구 모임이 있어요\u200b그중에 몇명만 코로나 방역수칙을 준수하며 만났습니다!\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n쌍룡계곡\n경상북도 문경시 농암면 내서리 쌍룡계곡\n\n\n\n\n\n \n\n\n\n위치는 쌍용계곡입니다 ㅎ\u200b경북 문경에 있습니다\u200b사람이 한적한 곳에 자릴 잡고 놀았습니다!\u200b수심이..어마어마해요..\u200b제가 팔을 쭉뻗고 들어가는데도 발이 닿지 않았으니까요 ㅎㄷㄷ\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n친구들과 장을 보구요! (상주 이마트)\u200b물건들을 챙겨서 계곡으로 향했습니다 ㅎ\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b보이시져 ㅋ 작은 폭포도 있고! 물가옆에서 자리잡은 우리!\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n곱창하고 돼지 목살도 구어주고요!\u200b너무 맛있어요! 역시 고기는 숯불이 제맛입니다!\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n진짜 맛있었어요 ㅎㅎㅎ 그리고 우측에 \u200b양송이에 치즈 넣어서 먹어봤는데요! 다음에도 해먹고 싶을만큼 멋진맛!\u2

In [734]:
count = 1
for data in final_data: 
    print(str(count) + '\n')
    print(data['댓글'])
    count+=1


1

[]
2

[]
3

[]
4

[]
5

[]
6

[]
7

[]
8

['아직도 접종 못하고 있는 1인. 부모 둘다 2회씩 맞았던것에 반해.', '이제 접종 속도는 더 빨라질겁니다 ㅎㅎ 조금만 힘내세요~', '아마 우리나라는 한동안 이 상태 일 것 같아요 <br>또 확진자가 1000명대로 늘어나서 1년 전 보다 사회적 거리 두기 단계가 훨씬 더 높아졌으니... <br>저도 지킨다 지킨다 했지만 좀 지치긴하네요.. 근데 끝나도 마스크는 당분간 쓰고 다닐 것 같아요<br>이제 안 쓰면 괜히 불안합니다 ㅠㅠ <br>백신 맞았긴 했는데 오히려 더 조심하게 되네요', '대국민 조별과제 + 탁상행정 == 내년까진 지속될 듯 합니다...', '포스팅 잘 읽고 갑니다! ○^○)&gt;<br>좋은 하루 되세요!<br>심심할 때 한 번 들러주세요~', '글 잘 보고 갑니다. 앞으로 소통하면서 지내요 ㅎㅎ!!', '저도 가고싶네요 (팔딱팔딱)', '진짜 언제쯤 일상의모습으로 돌아올지..', '코로나는 워낙에 변이도 많이 일어나는데다가..낮은 확률이지만 백신 접종 맞고 안타깝게 세상 떠나시는 분들 뉴스로도 봐서<br>진짜 충격 먹었던게 곧 전역예정이던 건강한 20대 병장 분도 백신 접종 맞고 그 소식이 떴더군요<br>몸이 건강하거나 젊어도 그 낮은 악운때문에 접종 받아도 세상이랑 하직하는 사례가 있어서 접종 받기가 매우 꺼려집니다']
9

[]
10

[]
11

[]
12

['가지마~~가지마 가지마~~~노래가 생각나네요 ㅋㅋ']
13

[]
14

['경실련 "SH공사 매입임대 2만 세대에 4조 원 투입...중단해야"<br><br>http://naver.me/FslJKWHs<br><br>경실련은 오늘(26일) 기자회견을 열고 SH공사가 지난 19년 동안 주택 2만 세대를 4조 원에 사들였지만, 적정성 검토 등이 부실해 부패세력의 불로소득 잔칫상이 되고 있다고 주장했습니다.<br><br>경실련은 SH공사가 하태경 의원실에 제출한 자료를 토대로 분석한 결과, 지난 200

In [728]:

pd.set_option('display.max_colwidth', None)
df_data = pd.DataFrame(final_data)
df_data['댓글']


0                                                                                                                                                                        []
1                                                                                                                                                                        []
2                                                                                                                                                                        []
3                                                                                                                                                                        []
4                                                                                                                                                                        []
                                                                                       ...                                                  

In [724]:
# df_data.to_csv('file.csv', encoding='utf-8-sig')

# Get info, Get reaction, Get Comment

In [698]:
def get_info(url_blog, params_blog): 
    tag_url = 'https://blog.naver.com/BlogTagListInfo.naver'
    tag_params = {
    'blogId': params_blog['blogId'],
    'logNoList': params_blog['logNo'],
    'logType': 'mylog'
}
    res = req.get(url_blog, headers = headers_default, params = params_blog)
    parsel = Selector(res.text)
    script =  parsel.xpath('/html/head/script[2]').getall()
    blogNo = script[0].split('var blogNo = ')[1].split('\'', 2)[1]
    soup = bs(res.text, 'html.parser')
    tmp  = {}
    try:
        soup.find('span', {'class': 'se-fs-'}).text
    except:
        try: 
            soup.find('p', {'class': 'se-text-paragraph'}).text
        except:
            try:
                soup.find('h3', {'class': 'se_textarea'}).text
            except:
                tmp['제목'] = soup.find('div', {'id': 'title_1'}).text
            else:
                tmp['제목'] = soup.find('h3', {'class': 'se_textarea'}).text
        else:
            tmp['제목'] = soup.find('p', {'class': 'se-text-paragraph'}).text
    else: 
        print(res.url)
        tmp['제목'] = soup.find('span', {'class': 'se-fs-'}).text
        
    tmp['작성자'] = soup.find('strong', {'class': 'nick'}).text
    try:
        soup.find('span', {'class': 'se_publishDate'}).text
    except:
        tmp['발행 시각'] = soup.find('p', {'class': '_postAddDate'}).text
    else:
        tmp['발행 시각'] = soup.find('span', {'class': 'se_publishDate'}).text
    
    try: 
        soup.find('div', {'class':'se-main-container'}).text
    except:
        try:
            soup.find('div', {'class': 'pcol2'}).find_all('p')
        except:
            tmp_p = ''
            p_list = soup.find_all('div', {'class': 'se_paragraph'})
            for p in p_list: 
                tmp_p = tmp_p + p.find('p', {'class': 'se_textarea'}).text  
            tmp['본문'] = tmp_p
        else:
            tmp_pg = ''
            pg_list = soup.find('div', {'class': 'pcol2'}).find_all('p')
            for pg in pg_list:
                tmp_pg = tmp_pg + pg.text
            tmp['본문'] = tmp_pg
                
    else: 
        tmp['본문'] = soup.find('div', {'class':'se-main-container'}).text
        
    res2 = req.get(tag_url, headers=headers_default, params=tag_params)
    js = res2.json()
    if js['taglist'] == []: 
        tmp['태그'] = ''
    else:
        tmp['태그']  = parse.unquote(js['taglist'][0]['tagName'])
    tmp['공감수'] = get_reaction(params_blog)
    tmp['댓글수'], tmp['댓글'] = get_comment(params_blog, blogNo)
    tmp['URL'] = res.url
    return tmp

In [370]:
url_reaction = 'https://blog.like.naver.com/v1/search/contents'
headers_json = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}

In [694]:
def get_reaction(params_blog, url2 = url_reaction, headers = headers_json):

    blogId= params_blog['blogId']
    logNo= params_blog['logNo']
#     print(blogId)
#     headers['referer'] = res_url+f'&sid1={sid}&oid={oid}&aid={aid}'
    soup = bs(res.text, 'html.parser')
    params_rct = {'suppress_response_codes': 'true', 'isDuplication': 'true'}
    params_rct['q'] = f'BLOG[{blogId}_{logNo}]'
#     print(params_rct)
    res2 = req.get(url2, headers=headers, params = params_rct)
    js = res2.json()
#     print(js)
    total = 0
    for react in js['contents'][0]['reactions']:
        total += react['count']
    return total

In [693]:
def get_comment(params_blog, blogNo, comment_page= 1, url3 = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json", headers = headers_json): 
    params= {
    'ticket': 'blog',
    'templateId': 'default',
    'pool': 'cbox9', 
    'lang': 'ko' ,
    'objectId': '',
    'pageSize': '50',
    'indexSize': '10',
    'groupId': '',
    'listType': 'OBJECT',
    'pageType': 'default',
    'page': '1',
    'initialize': 'true',
    'userType': '',
    'useAltSort': 'true',
    'replyPageSize': '10',
    'showReply': 'true'
    } 
    logNo = params_blog['logNo']
    blogId = params_blog['blogId']
    params['groupId'] = blogNo
    params['objectId'] = f'{blogNo}_201_{logNo}'
    headers['referer'] = f'https://blog.naver.com/PostView.naver?blogId={blogId}&logNo={logNo}&redirect=Dlog&widgetTypeCall=true&directAccess=false'
    res2 = req.get(url3, headers=headers, params=params)
    js = json.loads(res2.text.split('_callback(')[-1].split(');')[0])
    if js['code'] == '3300':  
        print(js['message'])
        count=0
        comment_list = []
#     number_per_page = len(js['result']['commentList'])
#     print(f"한페이지당 로드되는 댓글수: {number_per_page}")
    else:
        count = js['result']['count']['comment']
        comment_list = []
        for i in range(1, count//10+2): 
            if i != 1:
                params['page']= i
                res2 = req.get(url3, headers=headers, params=params)
                js = json.loads(res2.text.split('_callback(')[-1].split(');')[0]) 
            for comment in js['result']['commentList']:
                comment_list.append(comment['contents'])
        # reverse the order of list 
        comment_list = comment_list[::-1]
    return count, comment_list